# Desafío Spike - Oscar Mauriaca

Aqui una pequeña explicación del problema.

Bases de datos:

Costos marginales reales USD/MWh a nivel de barra y hora (cada fila es una barra y día-hora). costo_marginal_real.csv


Costos marginales programados USD/MWh (por el SIC) a nivel de barra y hora (cada fila es una barra y día-hora). costo_marginal_programado.csv


Base para entrenar modelos. base_para_predicción.csv


Nivel de observación: día-hora y subestación (cada fila es una subestación y día-hora).

nemotecnico_se: código subestación eléctrica

fecha, hora: año, mes, día y hora.

gen_eolica_total_mwh: generación eólica total por subestación en MWh

gen_geotermica_total_mwh: generación geotérmica total por subestación en MWh

gen_hidraulica_total_mwh: generación hidráulica total por subestación en MWh

gen_solar_total_mwh: generación solar total por subestación en MWh

gen_termica_total_mwh: generación térmica total por subestación en MWh

cmg_real: costo marginal real [USD/MWh] promedio por subestación

cmg_prog: costo marginal programado [USD/MWh] promedio por subestación

cmg_desv: desviación absoluta promedio entre costo marginal programado y costo marginal real

cmg_desv_pct: desviación porcentual promedio del costo marginal real con respecto al costo marginal programado

n_barras: número de barras conectadas aguas abajo de una subestación

demanda_mwh: retiros de energía [MWh] efectuados por compañías coordinadas bajo el sistema eléctrico

cap_inst_mw: capacidad de generación máxima instalada por subestación [MW]


In [1]:
import pandas as pd
import requests

In [2]:
url_costomg_real = 'https://storage.googleapis.com/sistema_electrico_chile/costo_marginal_real.csv'
url_costomg_prog = 'https://storage.googleapis.com/sistema_electrico_chile/costo_marginal_programado.csv'

In [3]:
df_real = pd.read_csv(url_costomg_real)
df_prog = pd.read_csv(url_costomg_prog)

In [8]:
df_real.tail()

,barra_mnemotecnico,barra_referencia_mnemotecnico,fecha,hora,costo_en_dolares,costo_en_pesos,nombre
4309946,BA04G065SE002G065,BA02T002SE032T002,2019-01-23,24,76.38,51.35944,BA S/E LA LAJA ENERGIA COYANCO BARRA 110KV
4309947,BA03R003SE023R003,BA02T002SE032T002,2019-01-23,24,87.71,58.97796,BA S/E SAN SEBASTIAN BARRA 66KV N° 1
4309948,BA01T005SE050T005,BA02T002SE032T002,2019-01-23,24,89.58,60.23538,BA S/E LICANTEN 66KV
4309949,BA01G111SE001G111,BA02T002SE032T002,2019-01-23,24,76.30,51.30565,BA S/E CENTRAL SANTA MARTA 220KV
4309950,BA12T025SE075T025,BA02T002SE032T002,2019-01-23,24,63.54,42.72557,BA S/E ESMERALDA 110KV


In [9]:
df_prog.tail()

,mnemotecnico_barra,nombre_barra,fecha,hora,costo
1192119,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,20,49.387641
1192120,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,21,50.186228
1192121,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,22,50.237093
1192122,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,23,49.969904
1192123,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,24,48.452660


In [16]:
duplicate_rows_prog = df_prog[df_prog.duplicated(['mnemotecnico_barra', 'nombre_barra', 'fecha', 'hora'])]

In [25]:
duplicate_rows_prog['mnemotecnico_barra'].unique()

array(['-', 'BA01T002SE036T002', 'BA02T003SE004T003', 'BA83L131SE134L131'],
      dtype=object)

In [29]:
df_prog.shape, duplicate_rows_prog.shape

((1192124, 5), (188229, 5))

In [30]:
df_prog.shape[0] - duplicate_rows_prog.shape[0]

1003895

In [31]:
df_prog.drop_duplicates(subset =['mnemotecnico_barra', 'nombre_barra', 'fecha', 'hora'], inplace = True )

In [19]:
df_real[(df_real['barra_mnemotecnico']=='BA83L131SE134L131') & (df_real['fecha']=='2019-06-06')]

,barra_mnemotecnico,barra_referencia_mnemotecnico,fecha,hora,costo_en_dolares,costo_en_pesos,nombre
133728,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,1,41.90,29.19508,BA S/E SIERRA GORDA 220KV - BP1
313184,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,2,41.63,29.00695,BA S/E SIERRA GORDA 220KV - BP1
492674,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,3,41.19,28.70037,BA S/E SIERRA GORDA 220KV - BP1
672506,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,4,40.05,27.90604,BA S/E SIERRA GORDA 220KV - BP1
852168,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,5,40.32,28.09417,BA S/E SIERRA GORDA 220KV - BP1
1031339,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,6,41.77,29.10450,BA S/E SIERRA GORDA 220KV - BP1
1210913,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,7,42.83,29.84309,BA S/E SIERRA GORDA 220KV - BP1
1390571,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,8,47.00,32.74866,BA S/E SIERRA GORDA 220KV - BP1
1569786,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,9,40.61,28.29624,BA S/E SIERRA GORDA 220KV - BP1
1749007,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,10,39.21,27.32074,BA S/E SIERRA GORDA 220KV - BP1


In [21]:
df_prog[(df_prog['mnemotecnico_barra']=='BA83L131SE134L131') & (df_prog['fecha']=='2019-06-06')]

,mnemotecnico_barra,nombre_barra,fecha,hora,costo
1192076,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,1,48.930459
1192077,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,2,47.723161
1192078,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,3,47.608940
1192079,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,4,46.289543
1192080,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,5,46.103264
1192081,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,6,47.664590
1192082,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,7,49.105734
1192083,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,8,48.973171
1192084,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,9,44.916320
1192085,BA83L131SE134L131,BA S/E SIERRA GORDA 220KV - BP1,2019-06-06,10,42.096565


In [43]:
df_prog.rename(columns = {'mnemotecnico_barra': 'barra_mnemotecnico'}, inplace = True)

In [95]:
result2 = pd.merge(df_real, df_prog, on = ['barra_mnemotecnico', 'fecha', 'hora'], how = 'left')

In [74]:
df_prog.drop(['nombre_barra'], axis = 1, inplace = True)

In [82]:
df_real.shape

(4309951, 7)

In [83]:
df_prog.shape

(1192124, 4)

In [85]:
df_real.shape[0] + df_prog.shape[0]

5502075

In [81]:
result[(result['barra_mnemotecnico']=='BA83L131SE134L131') & (result['fecha']=='2019-06-06')]

,barra_mnemotecnico,barra_referencia_mnemotecnico,fecha,hora,costo_en_dolares,costo_en_pesos,nombre,costo
29261,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,1,41.90,29.19508,BA S/E SIERRA GORDA 220KV - BP1,48.930459
29262,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,1,41.90,29.19508,BA S/E SIERRA GORDA 220KV - BP1,45.544456
67267,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,2,41.63,29.00695,BA S/E SIERRA GORDA 220KV - BP1,47.723161
67268,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,2,41.63,29.00695,BA S/E SIERRA GORDA 220KV - BP1,44.420703
105322,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,3,41.19,28.70037,BA S/E SIERRA GORDA 220KV - BP1,47.608940
105323,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,3,41.19,28.70037,BA S/E SIERRA GORDA 220KV - BP1,43.696282
143424,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,4,40.05,27.90604,BA S/E SIERRA GORDA 220KV - BP1,46.289543
143425,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,4,40.05,27.90604,BA S/E SIERRA GORDA 220KV - BP1,42.485318
181502,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,5,40.32,28.09417,BA S/E SIERRA GORDA 220KV - BP1,46.103264
181503,BA83L131SE134L131,BA02T002SE032T002,2019-06-06,5,40.32,28.09417,BA S/E SIERRA GORDA 220KV - BP1,42.314348


In [94]:
result

,barra_mnemotecnico,barra_referencia_mnemotecnico,fecha,hora,costo_en_dolares,costo_en_pesos,nombre,costo
0,BA01G021SE018G021,BA02T002SE032T002,2019-04-06,25,58.02,38.65989,BA S/E CENTRAL QUINTERO 220KV,54.292145
1,BA01T018SE001T018,BA02T002SE032T002,2019-04-06,25,63.60,42.37795,BA S/E NUEVA PICHIRROPULLI 220KV B1,60.620648
2,BA02T005SE055T005,BA02T002SE032T002,2019-04-06,25,62.78,41.83157,BA S/E SAN FRANCISCO DE MOSTAZAL 66KV T1,59.177969
3,BA01T002SE029T002,BA02T002SE032T002,2019-04-06,25,58.55,39.01304,BA S/E CENTRAL PEHUENCHE 220KV B1,54.566605
4,BA01R002SE041R002,BA02T002SE032T002,2019-04-06,25,60.39,40.23906,BA S/E SAN CRISTOBAL 110KV B1,56.086260
...,...,...,...,...,...,...,...,...
913383,BA05T005SE288T005,BA02T002SE032T002,2019-01-23,24,63.65,42.79953,BA S/E SUR 110KV,132.934674
913384,BA01L081SE066L081,BA02T002SE032T002,2019-01-23,24,73.18,49.20770,BA S/E FPC 154KV BP1,53.174872
913385,BA04T005SE071T005,BA02T002SE032T002,2019-01-23,24,67.20,45.18662,BA S/E OVALLE 66KV,144.613620
913386,BA01L079SE004L079,BA02T002SE032T002,2019-01-23,24,77.61,52.18652,BA S/E PUENTE ALTO (CMPC) 110KV,55.571525


In [87]:
result.shape[0] - df_real.shape[0] 

-3396563

In [96]:
result2

,barra_mnemotecnico,barra_referencia_mnemotecnico,fecha,hora,costo_en_dolares,costo_en_pesos,nombre,costo
0,BA01T005SE075T005,BA02T002SE032T002,2019-04-06,25,62.24,41.47176,BA S/E LORETO 15KV,NaN
1,BA01T005SE175T005,BA02T002SE032T002,2019-04-06,25,62.56,41.68498,BA S/E RETIRO 13.2KV,NaN
2,BA03R002SE002R002,BA02T002SE032T002,2019-04-06,25,60.04,40.00585,BA S/E ALTAMIRANO 12.5KV BP1,NaN
3,BA01R002SE012R002,BA02T002SE032T002,2019-04-06,25,61.65,41.07863,BA S/E CURACAVI 44KV B1,NaN
4,BA03R002SE013R002,BA02T002SE032T002,2019-04-06,25,59.55,39.67936,BA S/E EL MANZANO (ENEL DISTRIBUCIÓN) 23KV BP1,NaN
...,...,...,...,...,...,...,...,...
4322405,BA04G065SE002G065,BA02T002SE032T002,2019-01-23,24,76.38,51.35944,BA S/E LA LAJA ENERGIA COYANCO BARRA 110KV,NaN
4322406,BA03R003SE023R003,BA02T002SE032T002,2019-01-23,24,87.71,58.97796,BA S/E SAN SEBASTIAN BARRA 66KV N° 1,NaN
4322407,BA01T005SE050T005,BA02T002SE032T002,2019-01-23,24,89.58,60.23538,BA S/E LICANTEN 66KV,NaN
4322408,BA01G111SE001G111,BA02T002SE032T002,2019-01-23,24,76.30,51.30565,BA S/E CENTRAL SANTA MARTA 220KV,NaN


In [97]:
result2.dropna()

,barra_mnemotecnico,barra_referencia_mnemotecnico,fecha,hora,costo_en_dolares,costo_en_pesos,nombre,costo
13,BA01G021SE018G021,BA02T002SE032T002,2019-04-06,25,58.02,38.65989,BA S/E CENTRAL QUINTERO 220KV,54.292145
16,BA01T018SE001T018,BA02T002SE032T002,2019-04-06,25,63.60,42.37795,BA S/E NUEVA PICHIRROPULLI 220KV B1,60.620648
20,BA02T005SE055T005,BA02T002SE032T002,2019-04-06,25,62.78,41.83157,BA S/E SAN FRANCISCO DE MOSTAZAL 66KV T1,59.177969
24,BA01T002SE029T002,BA02T002SE032T002,2019-04-06,25,58.55,39.01304,BA S/E CENTRAL PEHUENCHE 220KV B1,54.566605
28,BA01R002SE041R002,BA02T002SE032T002,2019-04-06,25,60.39,40.23906,BA S/E SAN CRISTOBAL 110KV B1,56.086260
...,...,...,...,...,...,...,...,...
4322387,BA05T005SE288T005,BA02T002SE032T002,2019-01-23,24,63.65,42.79953,BA S/E SUR 110KV,132.934674
4322390,BA01L081SE066L081,BA02T002SE032T002,2019-01-23,24,73.18,49.20770,BA S/E FPC 154KV BP1,53.174872
4322395,BA04T005SE071T005,BA02T002SE032T002,2019-01-23,24,67.20,45.18662,BA S/E OVALLE 66KV,144.613620
4322404,BA01L079SE004L079,BA02T002SE032T002,2019-01-23,24,77.61,52.18652,BA S/E PUENTE ALTO (CMPC) 110KV,55.571525


In [102]:
result3 = pd.merge(df_real, df_prog, on = ['barra_mnemotecnico', 'fecha', 'hora'], how = 'inner')
result3

,barra_mnemotecnico,barra_referencia_mnemotecnico,fecha,hora,costo_en_dolares,costo_en_pesos,nombre,costo
0,BA01G021SE018G021,BA02T002SE032T002,2019-04-06,25,58.02,38.65989,BA S/E CENTRAL QUINTERO 220KV,54.292145
1,BA01T018SE001T018,BA02T002SE032T002,2019-04-06,25,63.60,42.37795,BA S/E NUEVA PICHIRROPULLI 220KV B1,60.620648
2,BA02T005SE055T005,BA02T002SE032T002,2019-04-06,25,62.78,41.83157,BA S/E SAN FRANCISCO DE MOSTAZAL 66KV T1,59.177969
3,BA01T002SE029T002,BA02T002SE032T002,2019-04-06,25,58.55,39.01304,BA S/E CENTRAL PEHUENCHE 220KV B1,54.566605
4,BA01R002SE041R002,BA02T002SE032T002,2019-04-06,25,60.39,40.23906,BA S/E SAN CRISTOBAL 110KV B1,56.086260
...,...,...,...,...,...,...,...,...
913383,BA05T005SE288T005,BA02T002SE032T002,2019-01-23,24,63.65,42.79953,BA S/E SUR 110KV,132.934674
913384,BA01L081SE066L081,BA02T002SE032T002,2019-01-23,24,73.18,49.20770,BA S/E FPC 154KV BP1,53.174872
913385,BA04T005SE071T005,BA02T002SE032T002,2019-01-23,24,67.20,45.18662,BA S/E OVALLE 66KV,144.613620
913386,BA01L079SE004L079,BA02T002SE032T002,2019-01-23,24,77.61,52.18652,BA S/E PUENTE ALTO (CMPC) 110KV,55.571525
